In [2]:
from testutil import change_to_root

change_to_root()

Current working directory: /Users/David/Documents/GitHub/outage-dashboard/tests
Changed to project root directory: /Users/David/Documents/GitHub/outage-dashboard


In [2]:
from data_prep import DataPreparer
dataprep = DataPreparer(start_year=2016, end_year=2023, folder='datasets', outage_file_name='outage_data.csv', financial_file_name='ppe.xlsx',normalize=True)
dataprep.save_to_csv(folder='datasets')

Linking Data...
Reading finance data from: datasets/ppe.xlsx
The data file is missing the 'Company' column or does not have any quarter-year format columns; attempting to clean data...
Column 'SP_ENTITY_ID' removed successfully.
Data successfully unpivoted and normalized.
Quarter and Year columns created successfully.
Finance data filtered and company names cleaned successfully.
Data linked successfully.


In [4]:
from data_prep import FinancialDataTransformer

f = FinancialDataTransformer(financial_data_file_name='ppe.xlsx', folder='datasets',normalize=True)
f.get_financial_data()

Reading finance data from: datasets/ppe.xlsx
The data file is missing the 'Company' column or does not have any quarter-year format columns; attempting to clean data...
Column 'SP_ENTITY_ID' removed successfully.
Data successfully unpivoted and normalized.
Quarter and Year columns created successfully.
Finance data filtered and company names cleaned successfully.


,Company,Quarter,PP&E,Year
0,AT&T Inc.,Q1,47.132000,2000
1,"Frontier Communications Parent, Inc.",Q1,2.946976,2000
2,"Lumen Technologies, Inc.",Q1,2.232390,2000
3,Comcast Corporation,Q1,4.764600,2000
4,Verizon Communications Inc.,Q1,39.749000,2000
...,...,...,...,...
667,"Lumen Technologies, Inc.",Q4,19.606000,2023
668,Comcast Corporation,Q4,58.165000,2023
669,Verizon Communications Inc.,Q4,133.036000,2023
670,"T-Mobile US, Inc.",Q4,72.038000,2023


In [5]:
from data_prep import OutageDataProcessor

o = OutageDataProcessor(outage_file_name='outage_data.csv', start_year=2021, end_year=2023,folder='datasets')
o.get_outage_frequency()

,Company,Year,Quarter,Count
0,AT&T Inc.,2021,Q1,211
1,AT&T Inc.,2021,Q2,224
2,AT&T Inc.,2021,Q3,235
3,AT&T Inc.,2021,Q4,229
4,AT&T Inc.,2022,Q1,216
...,...,...,...,...
139,Verizon Communications Inc.,2022,Q4,327
140,Verizon Communications Inc.,2023,Q1,300
141,Verizon Communications Inc.,2023,Q2,311
142,Verizon Communications Inc.,2023,Q3,348


In [3]:
from visualization import PlotlyVisualizer

vis = PlotlyVisualizer(directory='datasets', filename='prepared_data.csv')
vis.plot_average_outage_vs_ppe(enable_interval=True)

In [4]:
vis.plot_granular_outage_vs_ppe(enable_interval=True)

In [5]:
vis.plot_outage_per_ppe_boxplot(include_grand_total=True)

In [6]:
vis.plot_outage_per_ppe_over_time(show_percentiles=True)